<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_V20_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-06-06 13:14:20


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','EQUITAS', 'SCHAEFFLER ', 'ADANITRANS', 'UJJIVAN']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
#stock_ids

In [4]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


In [5]:
import yfinance as yfin
def get_stock_v20(stock_symbol):
    #print(stock_symbol)
    stock_df = yfin.Ticker(stock_symbol).history(period='max')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Candle'] = 'R'
    stock_df.loc[stock_df['Close'] > stock_df['Open'], 'Candle'] = 'G'
    stock_df['Prev_Candle'] = stock_df['Candle'].shift(1)
    stock_df['Next_Candle'] = stock_df['Candle'].shift(-1)
    stock_df['Signal'] = 'NA'
    stock_df.loc[(stock_df['Candle'] != stock_df['Prev_Candle']) & (stock_df['Candle'] == 'G'), 'Signal'] = 'Start'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'NA'), 'Signal'] = 'End'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'Start'), 'Signal'] = 'Start/End'
    stock_df = stock_df[stock_df['Signal'] != 'NA']
    stock_df['Prev_Low'] = stock_df['Low'].shift(1)
    stock_df = stock_df[stock_df['Signal'] != 'Start']
    stock_df.loc[(stock_df['Signal'] == 'End'), 'Low'] = stock_df['Prev_Low']
    stock_df['Up%'] = round((stock_df['High'] - stock_df['Low'])*100/stock_df['Low'],2)
    stock_df = stock_df[stock_df['Up%'] > 20]
    stock_df['Current'] = yfin.Ticker(stock_symbol).history(period='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]['Close'].values[0]
    stock_df = stock_df[stock_df['Current'] < stock_df['Low']]
    stock_df['Stock'] = stock_symbol
    return stock_df

def get_v20():
    df_v20 = pd.DataFrame()
    for stock_id in stock_ids:
        #print(stock_id)
        df_tmp = get_stock_v20(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        #df_v20 = df_v20.append(df_tmp)
        df_v20 = pd.concat([df_v20, df_tmp])

    return df_v20.sort_index()

In [6]:
cols = ['Stock', 'Up%', '200_SMA', 'Close' ,'Criteria']
df_v20_prospects = get_v20()[cols]

In [7]:
df_v20_prospects_200SMA = df_v20_prospects[(df_v20_prospects['Close'] <= df_v20_prospects['200_SMA'])]

In [8]:
df_v20_prospects_V40N = df_v20_prospects[(df_v20_prospects['Criteria'] == 'AV40N') | (df_v20_prospects['Criteria'] == 'AV40')]

In [9]:
df_final_v20 = pd.concat([df_v20_prospects_V40N.reset_index(), df_v20_prospects_200SMA.reset_index()])
df_final_v20.set_index("Date", inplace = True)
df_final_v20.sort_index()

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2006-06-09 00:00:00+05:30,BAJAJHIND.NS,43.27,266.0,258.118103,AVR
2006-08-02 00:00:00+05:30,BAJAJHIND.NS,20.80,289.0,270.067627,AVR
2007-01-03 00:00:00+05:30,IBREALEST.NS,59.85,302.0,234.962723,NaN
2007-01-29 00:00:00+05:30,IBREALEST.NS,32.40,304.0,288.235596,NaN
2007-02-02 00:00:00+05:30,NETWORK18.NS,27.78,657.0,364.143036,NaN
...,...,...,...,...,...
2023-05-23 00:00:00+05:30,AWL.NS,33.52,566.0,488.700012,AVSP
2023-05-24 00:00:00+05:30,VALIANTORG.NS,20.22,573.0,552.450012,AVR
2023-06-13 00:00:00+05:30,SFL.NS,20.34,1241.0,1210.449951,AV40N


In [11]:
df_final_v20.sort_index().tail(20)

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2022-03-10 00:00:00+05:30,TTKPRESTIG.NS,20.75,921.0,850.384583,AV40N
2022-03-10 00:00:00+05:30,TTKPRESTIG.NS,20.75,921.0,850.384583,AV40N
2022-03-28 00:00:00+05:30,TATAELXSI.NS,20.52,5609.0,8919.786133,AV40N
2022-03-31 00:00:00+05:30,AAVAS.NS,21.73,2675.0,2569.449951,AVSP
2022-04-06 00:00:00+05:30,ALKYLAMINE.NS,20.53,3528.0,3324.038818,NaN
2022-04-06 00:00:00+05:30,VAIBHAVGBL.NS,29.36,591.0,498.352814,AVR
2022-05-09 00:00:00+05:30,CAMPUS.NS,24.04,379.0,378.899994,AAR
2022-05-23 00:00:00+05:30,INDIGOPNTS.NS,26.98,2064.0,1684.879395,AV40N
2022-05-23 00:00:00+05:30,INDIGOPNTS.NS,26.98,2064.0,1684.879395,AV40N


In [10]:
df_v20_prospects['Criteria'].value_counts()

Criteria
AVR      200
AAR       56
AVSP      45
AV40N     31
AV40       4
EXP50      1
Name: count, dtype: int64